Simple Vanilla Neural Net 

In [13]:
#Image classifer with logistic regression
#images shape(width,height,3(rgb))

###to knit to html ipython nbconvert --to html [NEWFILENAME].html
import numpy as np
import scipy 
import os
import matplotlib.pyplot as plot
import math
from scipy import ndimage

#from basicFunctions import crossEntropyLoss 
 

In [14]:
# Global Constants
side_length = 64
image_path = "./Logistic_Regression_Data/"
test_ratio = .3
epoch = 7000
learning_rate = .005
max_num_images = -1

In [15]:
def getImageArrays(path, side_length): #returns list of images arrays for a specified path
    image_names = os.listdir(path)
    image_names = image_names[:max_num_images]
    examples = []
    for image_name in image_names:
        if image_name.split(".")[-1] != "DS_Store":
            try:
                cur_image_path = path + image_name
                cur_image = np.array(ndimage.imread(cur_image_path,flatten=False))
                cur_array_resized = scipy.misc.imresize(cur_image,size=(side_length,side_length))

    #             img = Image.fromarray(cur_array_resized, 'RGB')
    #             img.save('my.png')
    #             img.show()
                cur_array_flattened = cur_array_resized.reshape((side_length*side_length*3)).T
                #print("new", cur_array_resized.shape)
                #cur_array = scipy.ndimage.imread(cur_image_path,flatten=False) #reads image as numpy array (lenght,height,3)
                #cur_array_resized = scipy.misc.imresize(cur_array,size=(side_length,side_length,3)) #resizes images to a uniform shape
                #print("old", cur_array_resized.shape)
                examples += [cur_array_flattened] 
            except ValueError:
                print("Error in creating examples",image_name)
                
    return examples

In [16]:
#create examples & labels
cow_images_path = image_path + "cows/"
notCow_image_path = image_path + "notcows/"

examples_cow = getImageArrays(cow_images_path, side_length)
labels_cow = np.ones(len(examples_cow))
examples_notCow = getImageArrays(notCow_image_path, side_length)
labels_notCow = np.zeros(len(examples_notCow))

examples_cow = np.array(examples_cow)
examples_notCow = np.array(examples_notCow)
#assert(examples_cow.shape[1:] == (side_length,side_length,3)), "examples_cow are invalid shape"
examples = np.concatenate((examples_cow,examples_notCow))
labels = np.concatenate((labels_cow,labels_notCow))
print("Number of Examples:",len(examples))
print("Number of Labels:",len(labels))


Error in creating examples notcow197.jpg
Error in creating examples notcow20.jpg
Error in creating examples notcow38.jpg
Error in creating examples notcow415.jpg
Number of Examples: 994
Number of Labels: 994


In [17]:
shuffled_indexing = np.random.permutation(labels.shape[0])
examples = examples[shuffled_indexing]
labels = labels[shuffled_indexing]

In [18]:
#seperate train and test examples
number_examples_test = int(len(examples)*test_ratio)
number_labels_test = int(len(labels)*test_ratio)

examples_test = examples[:number_examples_test]
examples_train = examples[number_examples_test:]
labels_test = labels[:number_labels_test]
labels_train = labels[number_labels_test:]
print("Number of training examples: ", examples_train.T.shape)
print("Number of test examples: ", examples_test.T.shape)
print(labels_train.shape)

Number of training examples:  (12288, 696)
Number of test examples:  (12288, 298)
(696,)


In [19]:
#reshape labels for future matrix operations
labels_train = np.reshape(labels_train,(1,len(labels_train)))
labels_test = np.reshape(labels_test,(1,len(labels_test)))

In [20]:
#flatten examples
# flattened_train_examples = examples_train.reshape(examples_train.shape[0], -1).T
# flattened_test_examples = examples_test.reshape(examples_test.shape[0], -1).T  
# print("flattened examples",flattened_test_examples.shape,flattened_train_examples.shape)
flattened_train_examples = examples_train.T
flattened_test_examples = examples_test.T

In [21]:
# Standardize color values of the image (decrease computational cost durring cross entropy)
standardized_train_examples = flattened_train_examples/255 #225 is the maximum rgb value/ This is done to decrease varaince in inputs thus more efficint
standardized_test_examples = flattened_test_examples/255
print("standardized",standardized_test_examples.shape,standardized_train_examples.shape)

standardized (12288, 298) (12288, 696)


In [22]:
def crossEntropyLoss(a,Y):
    num_of_examples = Y.shape[1] #len(Y) doesn't work, need 2nd dimesnion
    a = a.T
    loss = - (1.0 / num_of_examples) * (np.dot((1.0 - Y), np.log(1.0 - a)) + np.dot(Y, np.log(a)))
    return loss

In [23]:
# Testing cross entropy function
test_y = np.asarray([[1,1,1]])
test_x = np.asarray([[.8,.9,.4]])
print(crossEntropyLoss(test_x,test_y))


[[ 0.4149316]]


In [24]:
def sigmoid(x):
    output = 1.0/(1.0+np.exp(-x))
    cached_value = x
    cached_value = {"out_a":x}
    return output, cached_value

In [25]:
def relu(x,leaky=0):
    output = np.maximum(leaky,x)
#     cached_value = x
    cached_value = {"out_a":x}
    return output, cached_value
    

In [26]:
# To generate numbers randomly with numpy, we will use np.random.randn(dimension_x, dimension_y)

In [27]:
def initialize_params_shallow(input_layer, hidden_layer, output_layer):
    w1 = np.random.randn(hidden_layer,input_layer)
    w2 = np.random.randn(output_layer,hidden_layer)
    #np.random.randn gives numbers larger than 1 so: (vanishing/exploading gradients)
    w1,w2 = w1 * .01, w2 * .01
    
    #is it by one because of broadcasting 
    b1 = np.zeros((hidden_layer,1))
    b2 = np.zeros((output_layer,1))
    
    assert(w1.shape == (hidden_layer, input_layer))
    assert(w2.shape == (output_layer, hidden_layer))
    assert(b1.shape == (hidden_layer,1))
    assert(b2.shape == (output_layer,1))
    
    return {"w1":w1,"w2":w2,"b1":b1,"b2":b2}

In [28]:
np.random.seed(1)
params_shallow = initialize_params_shallow(3, 4, 2)
print("w1:", params_shallow["w1"])
print("w2:", params_shallow["w2"])
print("b1:", params_shallow["b1"])
print("b2:", params_shallow["b2"])

w1: [[ 0.01624345 -0.00611756 -0.00528172]
 [-0.01072969  0.00865408 -0.02301539]
 [ 0.01744812 -0.00761207  0.00319039]
 [-0.0024937   0.01462108 -0.02060141]]
w2: [[-0.00322417 -0.00384054  0.01133769 -0.01099891]
 [-0.00172428 -0.00877858  0.00042214  0.00582815]]
b1: [[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
b2: [[ 0.]
 [ 0.]]


#### test case for initialize_params_shallow using seed(1)
##### (3,4,2)
<table style:"width-80%">
    <tr>
        <td>
         <b> w1:</b> [[ 0.01624345 -0.00611756 -0.00528172]
         [-0.01072969  0.00865408 -0.02301539]
         [ 0.01744812 -0.00761207  0.00319039]
         [-0.0024937   0.01462108 -0.02060141]]
        </td>
        <td>
        w2: [[-0.00322417 -0.00384054  0.01133769 -0.01099891]
 [-0.00172428 -0.00877858  0.00042214  0.00582815]]
        </td>
    </tr>
    <tr>
    <td>
        b1: [[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
        </td>
        <td>
b2: [[ 0.]
 [ 0.]]
 </td>
     </tr>
</table>

In [29]:
def initialize_params_deep(input_layer, hidden_layers, output_layer):
#     np.random.seed(3) #non random random numbers (will get the same random numbers each time) used for testing
    layers = {} #dictionary of weights and biasis
    previous_layer = input_layer #sets the begining shape as shape of input layer
    for i in range(len(hidden_layers)):
        cur_layer = hidden_layers[i]
        weights = np.random.randn(cur_layer,previous_layer)
        weights *= .01 #because np.random.rand returns numbers too large, more efficeint durring gradient descensed 
        bias = np.zeros((cur_layer,1))
        cur_weights_key = "w" + str(i + 1) # i + 1 because humans like things indexed at 0
        cur_bias_key = "b" + str(i + 1)
        layers[cur_weights_key] = weights
        layers[cur_bias_key] = bias 
        
        assert(layers["w" + str(i + 1)].shape == (cur_layer, previous_layer))
        assert(layers["b" + str(i + 1)].shape == (cur_layer,1))
        
        previous_layer = cur_layer

    layers["w" + str(len(hidden_layers) + 1)] = np.random.randn(output_layer,previous_layer) * .01
    layers["b" + str(len(hidden_layers) + 1)] = np.zeros((output_layer,1))

    return layers

In [208]:
params_deep1 = initialize_params_deep(5,[4],3)
print(params_deep1)
for param in params_deep1:
    print(param, params_deep1[param].shape)


{'w1': array([[-0.01743141, -0.0059665 , -0.00588594, -0.00873882,  0.00029714],
       [-0.02248258, -0.00267762,  0.01013183,  0.00852798,  0.01108187],
       [ 0.01119391,  0.01487543, -0.01118301,  0.00845833, -0.0186089 ],
       [-0.00602885, -0.01914472,  0.01048148,  0.01333738, -0.00197415]]), 'b1': array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.]]), 'w2': array([[ 0.01774645, -0.00674728,  0.00150617,  0.00152946],
       [-0.01064195,  0.00437947,  0.01938978, -0.01024931],
       [ 0.00899338, -0.00154507,  0.01769627,  0.00483788]]), 'b2': array([[ 0.],
       [ 0.],
       [ 0.]])}
w1 (4, 5)
b1 (4, 1)
w2 (3, 4)
b2 (3, 1)


In [32]:
np.random.seed(3)
params_deep2 = initialize_params_deep(5,[4],3)
for param in params_deep2:
    print(param, params_deep2[param].shape)
    print(params_deep2[param])


w1 (4, 5)
[[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 (4, 1)
[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
w2 (3, 4)
[[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 (3, 1)
[[ 0.]
 [ 0.]
 [ 0.]]


#### Testcase initialize_params_deep 
##### seed(3) (5,[3],3)
<table style:"width-80%">
    <tr>
        <td>
w1 (4, 5)
[[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
 </td>
 <td>
b1 (4, 1)
[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
</td>
<tr>
<td>
w2 (3, 4)
[[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
</td>
<td>
b2 (3, 1)
[[ 0.]
 [ 0.]
 [ 0.]]
 </td>
 </tr>
 </table>

In [57]:
def linear_forward_prop(a,w,b): #a because a is from the past layer
    print("Before Transpose -- A:",a.shape,"W:",w.shape,"B:",b.shape)
    print("After Transpose -- A:",a.shape,"W:",w.shape,"B:",b.shape)
    z = np.dot(w,a) + b
    print("z:",z.shape)
#     print("z",z.shape,"a",a.shape[0],"w",w.shape[1],"b",b.shape)
#     assert(z.shape == (a.shape[0], w.shape[1] ))
    #     assert(z.shape == np.dot(a,w).shape)
    cached_inputs = {"in_a":a,"w":w,"b":b} #(a,w,b)
    return z,cached_inputs
    

In [58]:
np.random.seed(1)
X = np.random.randn(3,2)
w = np.random.randn(1,3)
b = np.random.randn(1) #(1,1) broadcasting
print("X",X,"w",w,"b",b,"z",z)
z,_ = linear_forward_prop(X,w,b)
print("X",X.shape,"w",w.shape,"b",b.shape,"z",z.shape)
print(z)

X [[ 1.62434536 -0.61175641]
 [-0.52817175 -1.07296862]
 [ 0.86540763 -2.3015387 ]] w [[ 1.74481176 -0.7612069   0.3190391 ]] b [-0.24937038] z [[ 3.26295337 -1.23429987]]
Before Transpose -- A: (3, 2) W: (1, 3) B: (1,)
After Transpose -- A: (3, 2) W: (1, 3) B: (1,)
z: (1, 2)
X (3, 2) w (1, 3) b (1,) z (1, 2)
[[ 3.26295337 -1.23429987]]


##### Test case for linear_foward_prop
##### seed(1) x=(3,2) w=(1,3) b=(1)
<table style:"width-80%">
    <tr>
        <td>
X (3, 2) w (1, 3) b (1,) z (2, 1)
</td>
</tr>
    <tr>
        <td>
[[ 3.26295337]
 [-1.23429987]]
 </td>
</tr>
</table>

In [35]:
def singleFowardPropagation(X,w,b,activationType,leaky=0): #TODO change x to a
#     cached_inputs = {"w":None,"x":None,"b":None,"z":None}
    z,cached_values = linear_forward_prop(X,w,b)
    if activationType == "sigmoid":
        a, activation_cache = sigmoid(z)
    if activationType == "relu":
        a, activation_cache = relu(z,leaky=leaky)
#     final_cache = c
    cached_values.update(activation_cache)
    return a, cached_values  #add asserts 

In [36]:
np.random.seed(2)
X = np.random.randn(3,2)
w = np.random.randn(1,3)
b = np.random.randn(1,1) #(1,1) broadcasting
a,cached_values = singleFowardPropagation(X,w,b,"relu")
print("Relu A",a)

a,cached_values = singleFowardPropagation(X,w,b,"sigmoid")
print("Sigmoud A",a)
# print(X,w,b)

Before Transpose -- A: (3, 2) W: (1, 3) B: (1, 1)
After Transpose -- A: (2, 3) W: (3, 1) B: (1, 1)
z: (2, 1)
Relu A [[ 3.43896131]
 [ 0.        ]]
Before Transpose -- A: (3, 2) W: (1, 3) B: (1, 1)
After Transpose -- A: (2, 3) W: (3, 1) B: (1, 1)
z: (2, 1)
Sigmoud A [[ 0.96890023]
 [ 0.11013289]]


Relu A [[ 3.43896131]
 [ 0.        ]]
Sigmoud A [[ 0.96890023]
 [ 0.11013289]]

In [37]:
def fowardPropagate(X,params):
    print("len of params", len(params))
    num_layers = len(params) // 2 #// rounds down
    #assert len params is even
    a = X
    cached_values = []
    for i in range(1,num_layers):
        w = params["w" + str(i)]
        b = params["b" + str(i)]
        past_a = a
        a, cur_cached = singleFowardPropagation(past_a,w,b,"relu")
        cached_values += [cur_cached] #TODO: get foramt of cached values 
        
    w = params["w" + str(num_layers)]
    b = params["b" + str(num_layers)]
    a, cur_cached = singleFowardPropagation(a,w,b,"sigmoid")
    cached_values += [cur_cached] 
    #write assert statemt here
    
    return a, cached_values

In [76]:
np.random.seed(6)
X = np.random.randn(5,4)
w1 = np.random.randn(4,5)
b1 = np.random.randn(4,1)
w2 = np.random.randn(3,4)
b2 = np.random.randn(3,1)
w3 = np.random.randn(1,3)
b3 = np.random.randn(1,1)
params = {
    "w1":w1,
    "b1":b1,
    "w2":w2,
    "b2":b2,
    "w3":w3,
    "b3":b3
}
a, cached_values = fowardPropagate(X, params)
print("a:",a)
print("cached_values",cached_values)


len of params 6
Before Transpose -- A: (5, 4) W: (4, 5) B: (4, 1)
After Transpose -- A: (5, 4) W: (4, 5) B: (4, 1)
z: (4, 4)
Before Transpose -- A: (4, 4) W: (3, 4) B: (3, 1)
After Transpose -- A: (4, 4) W: (3, 4) B: (3, 1)
z: (3, 4)
Before Transpose -- A: (3, 4) W: (1, 3) B: (1, 1)
After Transpose -- A: (3, 4) W: (1, 3) B: (1, 1)
z: (1, 4)
a: [[ 0.03921668  0.70498921  0.19734387  0.04728177]]
cached_values [{'in_a': array([[-0.31178367,  0.72900392,  0.21782079, -0.8990918 ],
       [-2.48678065,  0.91325152,  1.12706373, -1.51409323],
       [ 1.63929108, -0.4298936 ,  2.63128056,  0.60182225],
       [-0.33588161,  1.23773784,  0.11112817,  0.12915125],
       [ 0.07612761, -0.15512816,  0.63422534,  0.810655  ]]), 'w': array([[ 0.35480861,  1.81259031, -1.3564758 , -0.46363197,  0.82465384],
       [-1.17643148,  1.56448966,  0.71270509, -0.1810066 ,  0.53419953],
       [-0.58661296, -1.48185327,  0.85724762,  0.94309899,  0.11444143],
       [-0.02195668, -2.12714455, -0.8344074

Before Transpose -- A: (5, 4) W: (4, 5) B: (4, 1)
After Transpose -- A: (5, 4) W: (4, 5) B: (4, 1)
z: (4, 4)
Before Transpose -- A: (4, 4) W: (3, 4) B: (3, 1)
After Transpose -- A: (4, 4) W: (3, 4) B: (3, 1)
z: (3, 4)
Before Transpose -- A: (3, 4) W: (1, 3) B: (1, 1)
After Transpose -- A: (3, 4) W: (1, 3) B: (1, 1)
z: (1, 4)
a: [[ 0.03921668  0.70498921  0.19734387  0.04728177]]

In [202]:
def linearBackprop(cached_values, dZ):
    a = cached_values["in_a"]
    w = cached_values["w"]
    b = cached_values["b"]
    number_examples = a.shape[1]
    dW = (1.0/number_examples) * np.dot(dZ,a.T)
    dB = (1.0/number_examples) * np.sum(dZ, axis = 1, keepdims= True)
    dA = np.dot(w.T, dZ)
#write asserts for dW dB & dA TODO
    return {"dW":dW,"dB":dB,"dA":dA}
    
    

In [203]:
np.random.seed(1)
dZ = np.random.randn(1,2)
a = np.random.randn(3,2)
w = np.random.randn(1,3)
b = np.random.randn(1,1)
cached_values = {"w": w, "in_a":a,"b":b}
linearBackprop(cached_values, dZ)

{'dA': array([[ 0.51822968, -0.19517421],
        [-0.40506361,  0.15255393],
        [ 2.37496825, -0.89445391]]),
 'dB': array([[ 0.50629448]]),
 'dW': array([[-0.10076895,  1.40685096,  1.64992505]])}

{'dA': array([[ 0.51822968, -0.19517421],
        [-0.40506361,  0.15255393],
        [ 2.37496825, -0.89445391]]),
 'dB': array([[ 0.50629448]]),
 'dW': array([[-0.10076895,  1.40685096,  1.64992505]])}

In [204]:
def relu_backward(dA, z):
    dZ = np.array(dA, copy=True) #convert dA to np array
    dZ[z <= 0] = 0
    return dZ
    

In [205]:
dZ = relu_backward([-1,3,2], np.array([2,-1,0]))
print(dZ)

[-1  0  0]


In [206]:
def sigmoid_backward(dA, z):
    s = 1/(1 + np.exp(-z))
    dz = dA * s * (1-s) 
    
    return dz

In [182]:
def backwards_activation(dA, cached_values, activation_type):
    if activation_type == "sigmoid":
        dZ = sigmoid_backward(dA, cached_values["z"])
    if activation_type == "relu":
        dZ = relu_backward(dA, cached_values["z"])
    gradients = linearBackprop(cached_values, dZ)
    return gradients
    
    #todo write optimizer function
        
        
    

In [183]:
# test_backwards_activation():
# returns input parameters of backwards_activation
np.random.seed(2)
dA = np.random.randn(1,2)
a = np.random.randn(3,2)
w = np.random.randn(1,3)
b = np.random.randn(1,1)
z = np.random.randn(1,2)
cached_values = {"w": w, "in_a":a,"b":b, "z":z}
backwards_activation(dA, cached_values, activation_type="relu")
backwards_activation(dA, cached_values, activation_type="sigmoid")
#working


LINERAR BACKPROP
w.T (3, 1)
dZ (1, 2)
LINERAR BACKPROP
w.T (3, 1)
dZ (1, 2)


{'dA': array([[ 0.11017994,  0.01105339],
        [ 0.09466817,  0.00949723],
        [-0.05743092, -0.00576154]]),
 'dB': array([[-0.05729622]]),
 'dW': array([[ 0.10266786,  0.09778551, -0.01968084]])}

For Relu:
{'dA': array([[ 0.44090989,  0.        ],
        [ 0.37883606,  0.        ],
        [-0.2298228 ,  0.        ]]),
 'dB': array([[-0.20837892]]),
 'dW': array([[ 0.44513824,  0.37371418, -0.10478989]])} 
 
For Sigmoid:
{'dA': array([[ 0.11017994,  0.01105339],
        [ 0.09466817,  0.00949723],
        [-0.05743092, -0.00576154]]),
 'dB': array([[-0.05729622]]),
 'dW': array([[ 0.10266786,  0.09778551, -0.01968084]])}

In [200]:
def backprop(a, cached_values, y):
    gradients = {}
    num_layers = len(cached_values)
    y = y.reshape(a.shape)

    dFinalA = -(np.divide(y,a) - np.divide(1-y, 1-a)) #dervivative of output of foward pass 
    backwards_activation_output_0 = backwards_activation(dFinalA, cached_values[-1],"sigmoid")
    
    gradients["dW" + str(num_layers)] = backwards_activation_output_0["dW"]
    gradients["dB" + str(num_layers)] = backwards_activation_output_0["dB"]
    gradients["dA" + str(num_layers - 1)] = backwards_activation_output_0["dA"]
    
    for i in reversed(range(num_layers - 1)):
        past_layer = cached_values[i]
        cur_dLayer = backwards_activation(gradients["dA" + str(i + 1)], past_layer, "relu")
        gradients["dW" + str(i + 1)] = cur_dLayer["dW"]
        gradients["dB" + str(i + 1)] = cur_dLayer["dB"]
        gradients["dA" + str(i)] = cur_dLayer["dA"]
    return gradients

In [207]:
np.random.seed(3)
a = np.random.randn(1, 2)
a1 = np.random.randn(4, 2)
b1 = np.random.randn(3, 4)
w1 = np.random.randn(3, 1)
z1 = np.random.randn(3, 2)
cache1 = [{"in_a" : a1,"b" : b1,"w" : w1, "z" : z1}]
a2 = np.random.randn(3, 2)
b2 = np.random.randn(1, 1)
w2 = np.random.randn(1, 3)
z2 = np.random.randn(1, 2)
cache2 = [{"in_a" : a2,"b" : b2,"w" : w2, "z" : z2}]
megaCache = cache1 +  cache2
y = np.array([[1,0]])
# print("y",y)
# print(megaCache[0]["in_a"],"in_a1")
# print(megaCache[0]["b"],"b1")
# print(megaCache[0]["w"],"w1")
# print(megaCache[0]["z"],"z1")
# print(megaCache[1]["in_a"],"in_a2")
# print(megaCache[1]["b"],"b2")
# print(megaCache[1]["w"],"w2")
# print(megaCache[1]["z"],"z2")


grads = backprop(a, megaCache,y)
print("dA0", grads["dA0"].shape)
print("dW1", grads["dW1"].shape)
print("dB1", grads["dB1"].shape)
print("dW2", grads["dW2"].shape)
print("dB2", grads["dB2"].shape)
print("dA1", grads["dA1"].shape)
print(grads)

dA0 (1, 2)
dW1 (3, 4)
dB1 (3, 1)
dW2 (1, 3)
dB2 (1, 1)
dA1 (3, 2)
{'dW2': array([[-0.39202432, -0.13325855, -0.04601089]]), 'dB2': array([[ 0.15187861]]), 'dA1': array([[-0.14175655,  0.48317296],
       [ 0.01663708, -0.05670698],
       [ 0.20472941, -0.69781407]]), 'dW1': array([[-0.45019464, -0.08570497, -0.15147489, -0.11528942],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.65018572,  0.1237779 ,  0.21876495,  0.16650473]]), 'dB1': array([[ 0.24158648],
       [ 0.        ],
       [-0.34890704]]), 'dA0': array([[ 0.        ,  1.43247838]])}


In [ ]:
def optimizer(params,examples,learing_rate,epoch):
    for i in range(epoch):
        for X,y in examples:
            a, cache = fowardPropagate(X,params)
            grads = backprop(a,params,cache,Y)
            for key,value in params:
                if key[1] != "A": 
                    cur_grad = grads["d" + key]
                    value = value - (learning_rate * grad)                    